In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from datetime import datetime
plt.style.use('ggplot')
pd.options.display.max_columns = 20
pd.options.display.max_rows = 30

In [ ]:
df = pd.read_csv("../input/datagolf/data_file.csv")
scoreDF = pd.read_csv("../input/scoresummary/Score Summary.csv")

In [ ]:
#Adjusting time values based on a time zone offset
df['shotTime'] = df['shotTime'] + df['shotTimeZoneOffset']

In [ ]:
scoreDF = scoreDF[[
#     'scorecardSummaries/id',
#  'scorecardSummaries/customerId',
#  'scorecardSummaries/playerProfileId',
#  'scorecardSummaries/scoreType',
 'scorecardSummaries/courseName',
 'scorecardSummaries/startTime',
#  'scorecardSummaries/endTime',
#  'scorecardSummaries/roundInProgress',
 'scorecardSummaries/strokes',
 'scorecardSummaries/handicappedStrokes',
 'scorecardSummaries/scoreWithHandicap',
 'scorecardSummaries/scoreWithoutHandicap',
#  'scorecardSummaries/holesCompleted',
#  'scorecardSummaries/roundType
]].copy()

In [ ]:
#Removing columns that cannot be analyzed
df = df[['holeNumber',
'id',
 'scorecardId',
 'playerProfileId',
 'shotTime',
 'shotOrder',
#  'shotTimeZoneOffset',
 'clubId',
#  'autoShotType',
#  'startLoc',
#  'endLoc',
 'meters',
#  'shotSource',
 'shotType']].copy()

In [ ]:
#Renaming columns from the scores dataset
scoreDF = scoreDF.rename(columns = { 'scorecardSummaries/courseName' : 'Course Name',
                                     'scorecardSummaries/startTime' : 'Shot Time',
                                     'scorecardSummaries/strokes': 'Strokes',
                                     'scorecardSummaries/handicappedStrokes' : 'Handicapped Strokes',
                                     'scorecardSummaries/scoreWithHandicap' : 'Score With Handicap',
                                     'scorecardSummaries/scoreWithoutHandicap': 'Score Without Handicap',})

In [ ]:
scoreDF['Shot Time'] = pd.to_datetime(scoreDF['Shot Time']).dt.date

In [ ]:
pd.options.display.max_rows = 30
#Removing the built in indexing on the right
df.style.hide(axis = "index")

#Renaming columns
df = df.rename(columns={'holeNumber': 'Hole Number',
                       'shotTime': 'Shot Time',
                       'shotOrder': 'Shot Order',
                       'meters': 'Distance Hit (yd)',
                       'clubId': 'Club Type'})


In [ ]:
#Converting shot time from unix timestamp to readable time
df['Shot Time'] = df['Shot Time'] // 1000
df['Shot Time'] = df['Shot Time'].apply(datetime.fromtimestamp)
df['Shot Time'] = pd.to_datetime(df['Shot Time']).dt.date

In [ ]:
#Find the cells with null values
df.isna().sum()

In [ ]:
#Find the rows that are duplicates of each other
df.loc[df.duplicated()]

In [ ]:
df.sort_values(by=['Shot Time', 'Hole Number'])

In [ ]:
#Find replace all the club ID's with a default value
clubMapping = {
    282646411: "Driver",
    282646416: "7W",
    282646471: "PW",
    282646456: "7i",
    282646486: "LW",
    282646481: "SW",
    449758208: "5W",
    282646466: "9i",
    304451726: "5H",
    312362566: "GW",
    282646491: "Putter",
    282646461: "8i",
    282646451: "6i",
    449758210: "4H",
    282646426: "3W",
    282646446: "5i",
    0: "Unknown"
}

In [ ]:
#Maps the club id value to an club name
df['Club Type'] = df['Club Type'].map(clubMapping)

In [ ]:
#Find all the unique club ID's
df['Club Type'].unique()
filteredData = df[df['Club Type'] != 'Unknown'].copy()

In [ ]:
#Converts distance hit from meters to yards
filteredData['Distance Hit (yd)'] = filteredData['Distance Hit (yd)'] * 1.09361
filteredData = filteredData.round(2).copy()

In [ ]:
#Create a dictionary containing information on all the distance hit by a specific club
clubDistanceDict = {}
for club in filteredData['Club Type'].unique():
    clubDistance = filteredData[filteredData['Club Type'] == club]['Distance Hit (yd)'].tolist()
    clubDistanceDict[club] = clubDistance    

In [ ]:
#Removing Outliers from the dataset
for club, distancesHit in clubDistanceDict.items():
    
    #Identifying quartiles and interquartile range
    Q1 = np.percentile(distancesHit, 25)
    Q3 = np.percentile(distancesHit, 75)
    IQR = Q3 - Q1
    
    #Determining the upper and lower bound with IQR
#     lowerBound = Q1 - 1.5 * IQR
#     upperBound = Q3 + 1.5 * IQR
    
    #Determing the upper and lower bounds with standard deviation 
    mean = sum(distancesHit) / len(distancesHit)
    stdDev = np.std(distancesHit)
    lowerBound = mean - 2.5 * stdDev
    upperBound = mean + 2.5 * stdDev
    
    outliers = []
    #If a value is outside either the lower or upper bound add them to a list
    for distance in distancesHit:
        if distance < lowerBound or distance > upperBound:
            outliers.append(distance)
    print(club, " " ,outliers)
    for dfDistanceHit in filteredData[filteredData['Club Type'] == club]['Distance Hit (yd)']:
        if dfDistanceHit in outliers:
            rowIndex = filteredData.index[filteredData['Distance Hit (yd)'] == dfDistanceHit]
            filteredData.drop(index= rowIndex, inplace=True)
    
    print("sample size:", len(distancesHit), "outlier ratio:", round(len(outliers)/len(distancesHit), 4) * 100, "%")
    print(club, "mean:", round(mean,2), " stdDev:" , round(stdDev * 1,2))
    print()
    

In [ ]:
filteredData.shape

In [ ]:
ax = df['Distance Hit (yd)'].plot(kind = 'kde', title = 'Distance Hit (yd)')

ax.set_xlabel('Distance (yd)')
ax.set_xlim(0,250)
plt.show()

In [ ]:

sns.histplot(data = df, x = 'Distance Hit (yd)').set(title = "Distance Hit Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(19, 5))


clubType = filteredData['Club Type']
DistanceHit = filteredData['Distance Hit (yd)']

avgDistanceHit = filteredData.groupby('Club Type')['Distance Hit (yd)'].mean().sort_values(ascending = False)

# Plot the barplot
sns.barplot(x=clubType, y=DistanceHit, order=avgDistanceHit.index)

plt.title("Average Distance Hit For Each Golf Club")
plt.show()


In [ ]:
plt.figure(figsize=(15, 5))

clubCount = filteredData['Club Type'].value_counts()

ax = sns.barplot(x=clubCount.index, y=clubCount)

# Add the numerical values on top of the bars
for index, timesUsed in enumerate(clubCount):
    ax.text(index, timesUsed, str(timesUsed), ha='center', va='bottom')

plt.show()


In [ ]:
plt.figure(figsize = (15,5))
clubData = filteredData[filteredData['Club Type'] == 'Driver']

averageClubData = clubData.groupby('Shot Time')['Distance Hit (yd)'].mean()
sns.scatterplot(x = averageClubData.index,y = averageClubData.values)
plt.show()
